In [1]:
import requests
from bs4 import BeautifulSoup as BS, element as BS_element
import lxml
import pandas as pd

In [2]:
%%time
# rutor_url = 'http://5.45.84.133/new/'
rutor_url = 'http://rutor.info/new'
host = '/'.join(rutor_url.split('/', maxsplit=3)[:3])
response = requests.get(rutor_url)
response.status_code

CPU times: total: 0 ns
Wall time: 409 ms


200

In [3]:
soup = BS(response.content, 'lxml')
main_table = soup.find('div', id='index')
dependent_tables: tuple[BS_element.Tag] = tuple(main_table.find('h2').find_next_siblings())

In [4]:
def _get_from_table(table: BS_element.Tag, group_name: str) -> pd.DataFrame:
    elements: tuple[BS_element.Tag] = tuple(table.find_all('td'))
    BS_data, BS_name, BS_size, BS_pirs = elements[0],elements[1],elements[-2],elements[-1]
    
    data = BS_data.get_text(strip=True)
    
    BS_name: BS_element.Tag = tuple(BS_name.find_all('a'))[-1]
    name = BS_name.get_text(strip=True)
    href = host + BS_name.get('href')

    size = BS_size.get_text(strip=True)
    
    up, down = tuple(map(int, BS_pirs.stripped_strings))
    
    return pd.DataFrame({
        'group': group_name,
        'data': data,
        'name': name,
        'size': size,
        'up': up,
        'down': down,
        'href': href
    }, index=[0])
    
    

def _get_table(elements: tuple[BS_element.Tag, BS_element.Tag]) -> list[pd.DataFrame]:
    bs_name, bs_table = elements
    group_name = bs_name.find('a').get_text(strip=True)
    group_tables: tuple[BS_element.Tag] = tuple(bs_table.find('tr', class_='backgr').find_next_siblings('tr'))
    
    ap_dataframe = []
    for item in group_tables:
        ap_dataframe.append(_get_from_table(item, group_name))
    
    return ap_dataframe
    
    
rearranged_tables: tuple[tuple[BS_element.Tag, BS_element.Tag]] = tuple(zip(dependent_tables[0::2], dependent_tables[1::2]))
ap_dataframe = []
for item in rearranged_tables:
    ap_dataframe.append(_get_table(item))

ap_dataframe = sum(ap_dataframe, [])
df = pd.concat(ap_dataframe, ignore_index=True, sort= False)

In [5]:
test = df['group'].unique()
print(test)
interesting = test[[0, 3, 6, 7, 10]]

['Зарубежные фильмы' 'Наши фильмы' 'Научно-популярные фильмы'
 'Зарубежные сериалы' 'Наши сериалы' 'Телевизор' 'Мультипликация' 'Аниме'
 'Иностранные релизы' 'Музыка' 'Игры' 'Софт' 'Спорт и Здоровье' 'Юмор'
 'Хозяйство и Быт' 'Книги' 'Другое']


In [6]:
my_df = df[df['group'].isin(interesting)]

cut = 40
my_df = my_df[(my_df['up'] >= cut) | (my_df['down'] >= cut)].reset_index(drop='index')
pd.set_option('display.max_colwidth', 1000)
display(my_df[my_df.columns[:-1]])

,group,data,name,size,up,down
0,Зарубежные фильмы,24 Июн 23,Проникновение / Private Property (2022) WEB-DLRip от ELEKTRI4KA | D | Локализованная версия,1.46 GB,125,47
1,Зарубежные фильмы,24 Июн 23,Проникновение / Private Property (2022) WEB-DL 1080p от ELEKTRI4KA | D | Локализованная версия,3.56 GB,83,96
2,Зарубежные фильмы,23 Июн 23,"Машина / The Machine (2023) WEB-DL 1080p от New-Team | HDRezka Studio, Jaskier",7.78 GB,131,39
3,Зарубежные фильмы,23 Июн 23,Машина / The Machine (2023) WEB-DLRip от New-Team | Jaskier,1.46 GB,108,36
4,Зарубежные фильмы,23 Июн 23,Стоп-слово / Sanctuary (2022) WEB-DL 1080p | L | AlphaProject,6.22 GB,151,64
5,Зарубежные сериалы,24 Июн 23,Черное зеркало / Black Mirror [S01-06 + Bandersnatch] (2011-2023) WEB-DLRip | LostFilm,21.30 GB,23,105
6,Зарубежные сериалы,24 Июн 23,Укрытие / Бункер / Silo [01x01-09 из 10] (2023) WEB-DLRip | Невафильм,7.53 GB,135,262
7,Зарубежные сериалы,24 Июн 23,Секретное вторжение / Secret Invasion [01x01 из 06] (2023) WEB-DL 1080p от Scarabey | D | Red Head Sound,2.36 GB,255,48
8,Зарубежные сериалы,23 Июн 23,Ходячие мертвецы: Мертвый город / The Walking Dead: Dead City [01x01-02 из 06] (2023) WEB-DL 1080p от NewComers | P,5.97 GB,57,40
9,Зарубежные сериалы,23 Июн 23,Ходячие мертвецы: Мертвый город / The Walking Dead: Dead City [01x01-02 из 06] (2023) WEBRip | NewComers,842.46 MB,58,39


In [8]:
id = [0, 16]


my_df.loc[id, 'href']

0       http://rutor.info/torrent/930896/proniknovenie_private-property-2022-web-dlrip-ot-elektri4ka-d-lokalizovannaja-versija
16    http://rutor.info/torrent/879664/thehunter-call-of-the-wild-complete-collection-v-2546982-dlcs-2017-pc-repack-ot-fitgirl
Name: href, dtype: object